# TowerGuard ML Pipeline Validation Notebook

**Project:** TowerGuard - Water Tower Environmental Health Monitoring  
**Track:** Data-Driven Impact Measurement (Wangari Maathai Hackathon)  
**Objective:** Validate end-to-end ML pipeline (NDVI → Features → Scoring) on 3 real water towers

This notebook demonstrates:
1. Feature extraction from Sentinel-2 satellite data
2. Environmental feature gathering (rainfall, temperature, elevation)
3. Rule-based health score computation
4. Visualization and interpretation of results

**Test Sites:** Mau, Aberdare, Mt. Elgon

## Section 1: Set Up Development Environment

Import essential libraries and configure logging for reproducibility.

In [ ]:
import sys
import os
from pathlib import Path

# Add backend/ml to path for imports
sys.path.insert(0, str(Path().resolve().parent.parent / 'ml'))

import numpy as np
import pandas as pd
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML pipeline modules
from utils import logger, setup_logger, build_feature_vector_from_site_features, FEATURE_FIELDS
from features import SiteFeatures, extract_features_for_site
from scoring import compute_health_score, create_prediction_for_site_features, SCORING_RULES, HEALTH_CATEGORIES

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configure logging
logger = setup_logger(name="validation", log_dir=str(Path().resolve().parent.parent / 'logs'))
logger.info("=" * 80)
logger.info("Starting TowerGuard ML Pipeline Validation")
logger.info("=" * 80)

print("✓ Environment configured successfully")
print(f"✓ Logging to: backend/logs/ml_pipeline.log")

## Section 2: Define Test Sites Configuration

Define the 3 water towers for validation with mock features (in production, these would come from actual Sentinel-2 data and climate APIs).

In [ ]:
# Define test sites with mock features
# In production, these would use actual Sentinel-2 data and climate APIs

test_sites = {
    'mau': {
        'site_id': 'mau',
        'name': 'Mau Water Tower',
        'latitude': -0.5,
        'longitude': 35.0,
        'description': 'Large highland forest complex in Western Kenya',
        # Mock features (in production: from NDVI computation and climate APIs)
        'features': {
            'ndvi_mean': 0.62,        # Good vegetation
            'ndvi_std': 0.18,         # Meets low variance threshold
            'rainfall_mm': 1850.0,    # High rainfall
            'temp_mean_c': 18.5,      # Good temperature
            'elevation_m': 2400.0     # Within optimal range
        }
    },
    'aberdare': {
        'site_id': 'aberdare',
        'name': 'Aberdare Water Tower',
        'latitude': -0.35,
        'longitude': 36.8,
        'description': 'Mountain range forming part of the Eastern Rift Valley',
        'features': {
            'ndvi_mean': 0.55,        # Good vegetation
            'ndvi_std': 0.22,         # Slightly above threshold
            'rainfall_mm': 980.0,     # Moderate rainfall
            'temp_mean_c': 16.2,      # Within optimal range
            'elevation_m': 2700.0     # Within optimal range
        }
    },
    'mt_elgon': {
        'site_id': 'mt_elgon',
        'name': 'Mt. Elgon Water Tower',
        'latitude': 1.1,
        'longitude': 34.6,
        'description': 'Extinct volcano on Kenya-Uganda border',
        'features': {
            'ndvi_mean': 0.48,        # Below optimal vegetation
            'ndvi_std': 0.25,         # High variance
            'rainfall_mm': 1100.0,    # Adequate rainfall
            'temp_mean_c': 14.8,      # Cool but acceptable
            'elevation_m': 3100.0     # Above optimal elevation range
        }
    }
}

# Display test sites summary
print("=" * 80)
print("TEST SITES CONFIGURATION")
print("=" * 80)

for site_id, site_data in test_sites.items():
    print(f"\n📍 {site_data['name']} ({site_id.upper()})")
    print(f"   Location: {site_data['latitude']:.2f}°, {site_data['longitude']:.2f}°")
    print(f"   Description: {site_data['description']}")
    print(f"   Features:")
    for feat, value in site_data['features'].items():
        units = {'ndvi_mean': '', 'ndvi_std': '', 'rainfall_mm': 'mm', 'temp_mean_c': '°C', 'elevation_m': 'm'}
        print(f"      • {feat}: {value} {units[feat]}")

print("\n" + "=" * 80)

## Section 3: Feature Vector Construction & Validation

Build feature vectors from mock features and validate against schema constraints.

In [ ]:
from utils import validate_feature_vector

# Build feature vectors for all sites
print("FEATURE VECTOR CONSTRUCTION & VALIDATION")
print("=" * 80)

feature_vectors = {}

for site_id, site_data in test_sites.items():
    features_dict = site_data['features']
    
    # Build feature vector
    result = build_feature_vector_from_site_features(features_dict)
    feature_vectors[site_id] = result['feature_vector']
    
    # Validate
    validation = validate_feature_vector(result['feature_vector'])
    
    print(f"\n📊 {site_id.upper()}")
    print(f"   Feature Vector: {result['feature_vector']}")
    print(f"   Complete: {result['is_complete']}")
    print(f"   Valid: {validation['valid']}")
    
    if validation['warnings']:
        for warning in validation['warnings']:
            print(f"   ⚠️  {warning}")
    
    if validation['errors']:
        for error in validation['errors']:
            print(f"   ❌ {error}")

# Create feature dataframe for easy comparison
feature_df = pd.DataFrame(
    {site_id: feature_vectors[site_id] for site_id in test_sites.keys()},
    index=FEATURE_FIELDS
)

print("\n\nFEATURE MATRIX (all sites)")
print(feature_df.to_string())
print("\n" + "=" * 80)

## Section 4: Rule-Based Health Score Computation

Apply scoring rules to each site and compute health scores.

In [ ]:
print("\nDISPLAYING SCORING RULES")
print("=" * 80)
print("\nRules defined in scoring.py:\n")

for i, rule in enumerate(SCORING_RULES, 1):
    print(f"{i}. {rule.condition}")
    print(f"   Points if met: +{rule.points_if_met}")
    print()

# Compute health scores
predictions = {}

for site_id, site_data in test_sites.items():
    # Create SiteFeatures object
    features_obj = SiteFeatures(
        site_id=site_id,
        ndvi_mean=site_data['features']['ndvi_mean'],
        ndvi_std=site_data['features']['ndvi_std'],
        rainfall_mm=site_data['features']['rainfall_mm'],
        temp_mean_c=site_data['features']['temp_mean_c'],
        elevation_m=site_data['features']['elevation_m']
    )
    
    # Compute health score
    score_result = compute_health_score(features_obj)
    predictions[site_id] = score_result
    
    # Print results
    print(f"\n{'=' * 80}")
    print(f"SITE: {site_data['name'].upper()} ({site_id})")
    print(f"{'=' * 80}")
    
    category = score_result['category_name']
    score = score_result['health_score']
    raw_points = score_result['raw_points']
    max_points = score_result['max_points']
    
    print(f"\n📊 HEALTH SCORE: {score:.1%} ({category})")
    print(f"   Points: {raw_points:.0f} / {max_points} points\n")
    
    print("RULE BREAKDOWN:")
    for rule_result in score_result['rule_results']:
        status = "✓" if rule_result['met'] else "✗"
        rule_text = rule_result['rule']
        points = rule_result['points_earned']
        explanation = rule_result['explanation']
        
        print(f"  {status} {rule_text}")
        print(f"     → {explanation} ({points:.0f} pts)")

print("\n" + "=" * 80)

## Section 5: Prediction Output & Interpretation

Generate comprehensive prediction outputs with human-readable interpretations.

In [ ]:
# Create comprehensive predictions
full_predictions = {}

for site_id, site_data in test_sites.items():
    # Create SiteFeatures object
    features_obj = SiteFeatures(
        site_id=site_id,
        ndvi_mean=site_data['features']['ndvi_mean'],
        ndvi_std=site_data['features']['ndvi_std'],
        rainfall_mm=site_data['features']['rainfall_mm'],
        temp_mean_c=site_data['features']['temp_mean_c'],
        elevation_m=site_data['features']['elevation_m']
    )
    
    # Create full prediction with interpretation
    metadata = {
        'site_name': site_data['name'],
        'location': f"{site_data['latitude']:.2f}°, {site_data['longitude']:.2f}°",
        'description': site_data['description'],
        'data_sources': {
            'ndvi': 'Sentinel-2 (Mock)',
            'rainfall': 'WorldClim (Mock)',
            'temperature': 'WorldClim (Mock)',
            'elevation': 'SRTM (Mock)'
        }
    }
    
    prediction = create_prediction_for_site_features(features_obj, metadata)
    full_predictions[site_id] = prediction
    
    # Display interpretation
    print(prediction['explanation'])
    print("\n" + "=" * 80 + "\n")

# Save predictions to JSON files
output_dir = Path().resolve().parent.parent / 'docs' / 'examples'
output_dir.mkdir(parents=True, exist_ok=True)

for site_id, prediction in full_predictions.items():
    filepath = output_dir / f"prediction_{site_id}.json"
    with open(filepath, 'w') as f:
        json.dump(prediction, f, indent=2)
    print(f"✓ Saved prediction: {filepath}")

## Section 6: Comparative Analysis & Visualization

Visualize and compare health scores across the 3 water towers.

In [ ]:
# Create comparison dataframe
comparison_data = []

for site_id, prediction in full_predictions.items():
    site_data = test_sites[site_id]
    comparison_data.append({
        'Site': site_data['name'],
        'Health Score': prediction['health_score'],
        'Category': prediction['category'],
        'Raw Points': prediction['raw_points'],
        'NDVI Mean': prediction['features']['ndvi_mean'],
        'Rainfall (mm)': prediction['features']['rainfall_mm'],
        'Temp (°C)': prediction['features']['temp_mean_c'],
        'Elevation (m)': prediction['features']['elevation_m']
    })

comparison_df = pd.DataFrame(comparison_data)

print("\nCOMPARATIVE SUMMARY - ALL SITES")
print("=" * 100)
print(comparison_df.to_string(index=False))
print("=" * 100)

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('TowerGuard Water Tower Health Metrics Comparison', fontsize=16, fontweight='bold')

# Plot 1: Health Scores
ax1 = axes[0, 0]
colors = ['#d62728' if score < 0.4 else '#ff7f0e' if score < 0.6 else '#2ca02c' 
          for score in comparison_df['Health Score']]
bars1 = ax1.barh(comparison_df['Site'], comparison_df['Health Score'], color=colors, alpha=0.8)
ax1.set_xlabel('Health Score (0-1)', fontweight='bold')
ax1.set_title('Overall Health Scores')
ax1.set_xlim(0, 1)
for i, v in enumerate(comparison_df['Health Score']):
    ax1.text(v + 0.02, i, f'{v:.1%}', va='center', fontweight='bold')

# Plot 2: NDVI Mean
ax2 = axes[0, 1]
bars2 = ax2.bar(comparison_df['Site'], comparison_df['NDVI Mean'], color='green', alpha=0.7)
ax2.axhline(y=0.5, color='red', linestyle='--', linewidth=2, label='Rule Threshold (0.5)')
ax2.set_ylabel('NDVI Mean', fontweight='bold')
ax2.set_title('Vegetation Index (NDVI)')
ax2.set_ylim(0, 1)
ax2.legend()
ax2.tick_params(axis='x', rotation=45)

# Plot 3: Rainfall
ax3 = axes[1, 0]
bars3 = ax3.bar(comparison_df['Site'], comparison_df['Rainfall (mm)'], color='blue', alpha=0.7)
ax3.axhline(y=120, color='red', linestyle='--', linewidth=2, label='Rule Threshold (120 mm)')
ax3.set_ylabel('Rainfall (mm)', fontweight='bold')
ax3.set_title('Annual Rainfall')
ax3.legend()
ax3.tick_params(axis='x', rotation=45)

# Plot 4: Temperature & Elevation
ax4 = axes[1, 1]
x_pos = np.arange(len(comparison_df['Site']))
width = 0.35

# Normalize temperature and elevation for comparison
temp_normalized = (comparison_df['Temp (°C)'] - comparison_df['Temp (°C)'].min()) / (comparison_df['Temp (°C)'].max() - comparison_df['Temp (°C)'].min())
elev_normalized = (comparison_df['Elevation (m)'] - comparison_df['Elevation (m)'].min()) / (comparison_df['Elevation (m)'].max() - comparison_df['Elevation (m)'].min())

bars4a = ax4.bar(x_pos - width/2, temp_normalized, width, label='Temperature (normalized)', alpha=0.8)
bars4b = ax4.bar(x_pos + width/2, elev_normalized, width, label='Elevation (normalized)', alpha=0.8)

ax4.set_ylabel('Normalized Value', fontweight='bold')
ax4.set_title('Temperature & Elevation (Normalized)')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(comparison_df['Site'], rotation=45)
ax4.legend()
ax4.set_ylim(0, 1.1)

plt.tight_layout()
plt.savefig(output_dir / 'health_metrics_comparison.png', dpi=300, bbox_inches='tight')
print(f"\n✓ Saved visualization: {output_dir / 'health_metrics_comparison.png'}")
plt.show()

print("\n" + "=" * 80)

## Section 7: Feature Vector Visualization

Display feature vectors in matrix form with heatmap.

In [ ]:
# Create feature vector heatmap
fig, ax = plt.subplots(figsize=(10, 6))

# Normalize feature vectors for visualization (0-1 scale per feature)
feature_matrix = feature_df.copy()
for idx in feature_matrix.index:
    min_val = feature_matrix.loc[idx].min()
    max_val = feature_matrix.loc[idx].max()
    if max_val > min_val:
        feature_matrix.loc[idx] = (feature_matrix.loc[idx] - min_val) / (max_val - min_val)

# Create heatmap
sns.heatmap(
    feature_matrix,
    annot=feature_df.values,
    fmt='.3f',
    cmap='RdYlGn',
    cbar_kws={'label': 'Normalized Value'},
    ax=ax,
    vmin=0,
    vmax=1,
    linewidths=0.5,
    cbar=True
)

ax.set_title('Feature Vector Heatmap (Values shown, Color = Normalized)', fontsize=14, fontweight='bold')
ax.set_xlabel('Water Tower Sites', fontweight='bold')
ax.set_ylabel('Features', fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / 'feature_vectors_heatmap.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved visualization: {output_dir / 'feature_vectors_heatmap.png'}")
plt.show()

print("\nFEATURE VECTOR VALUES (Raw)")
print(feature_df.to_string())

## Section 8: Scoring Breakdown Comparison

Detailed per-rule comparison across all sites.

In [ ]:
print("\nSCORING BREAKDOWN BY SITE")
print("=" * 100)

# Create scoring breakdown table
scoring_data = []

for site_id, prediction in full_predictions.items():
    site_name = test_sites[site_id]['name']
    
    for rule_idx, rule_result in enumerate(prediction['rule_breakdown'].items()):
        rule_name, rule_data = rule_result
        scoring_data.append({
            'Site': site_name,
            'Rule': rule_name,
            'Value': rule_data['value'],
            'Met': '✓' if rule_data['met'] else '✗',
            'Points': rule_data['points']
        })

scoring_df = pd.DataFrame(scoring_data)

# Pivot to see rules as columns
scoring_pivot = scoring_df.pivot_table(
    index='Site',
    columns='Rule',
    values='Points',
    aggfunc='first'
)

print("\nPOINTS PER RULE (all sites)")
print(scoring_pivot.to_string())

# Summary by site
print("\n\nTOTAL POINTS BY SITE")
print("-" * 50)
for site_id, prediction in full_predictions.items():
    site_name = test_sites[site_id]['name']
    total_pts = sum(rule['points'] for rule in prediction['rule_breakdown'].values())
    health_score = prediction['health_score']
    category = prediction['category']
    print(f"{site_name:30} {total_pts:5.0f} pts ({health_score:.1%}) - {category}")

print("\n" + "=" * 100)

## Section 9: Validation Summary & Deliverables

Final validation results and artifacts summary.

In [ ]:
print("\n" + "=" * 100)
print("VALIDATION SUMMARY")
print("=" * 100)

print("\n✓ COMPLETED TASKS:")
print("  1. ✓ Set up development environment with all dependencies")
print("  2. ✓ Loaded and configured 3 water tower test sites (Mau, Aberdare, Mt. Elgon)")
print("  3. ✓ Built feature vectors with schema validation")
print("  4. ✓ Applied all 5 rule-based scoring rules")
print("  5. ✓ Computed health scores (0-1 scale) and categories")
print("  6. ✓ Generated human-readable interpretations")
print("  7. ✓ Created visualizations for comparative analysis")
print("  8. ✓ Validated feature completeness and ranges")

print("\n📊 VALIDATION RESULTS:")

for site_id, prediction in full_predictions.items():
    site_name = test_sites[site_id]['name']
    score = prediction['health_score']
    category = prediction['category']
    missing = len(prediction['missing_features'])
    
    print(f"\n  {site_name}:")
    print(f"    • Health Score: {score:.1%} ({category})")
    print(f"    • Missing Features: {missing}/5")
    if missing > 0:
        print(f"      {prediction['missing_features']}")

print("\n📁 DELIVERABLES (saved to docs/examples/):")

# List all output files
output_files = sorted(output_dir.glob('*'))
for filepath in output_files:
    print(f"  • {filepath.name}")

print("\n📝 PIPELINE COMPONENTS:")
print("  ✓ backend/ml/utils.py - Logging, feature vectors, validation")
print("  ✓ backend/ml/ndvi.py - Sentinel-2 NDVI computation")
print("  ✓ backend/ml/features.py - Multi-source feature extraction")
print("  ✓ backend/ml/scoring.py - Rule-based health scoring")
print("  ✓ backend/tests/test_ml_pipeline.ipynb - This validation notebook")
print("  ✓ docs/examples/feature_schema.json - Schema specification")

print("\n🎯 KEY INSIGHTS:")
print(f"  • Best performing site: {comparison_df.loc[comparison_df['Health Score'].idxmax(), 'Site']}")
print(f"    Score: {comparison_df['Health Score'].max():.1%}")
print(f"  • Needs attention: {comparison_df.loc[comparison_df['Health Score'].idxmin(), 'Site']}")
print(f"    Score: {comparison_df['Health Score'].min():.1%}")

print("\n" + "=" * 100)
print("VALIDATION COMPLETE")
print("=" * 100)

logger.info("=" * 80)
logger.info("TowerGuard ML Pipeline Validation Complete")
logger.info("=" * 80)